In [9]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

## Questions and Answers
1. The neural network regression model performed better than the baseline linear regression model, but the difference was relatively small (MSE decreased by ~0.01).
2. The cost function is "adam", or the squared error using stochastic gradient descent.
3. See table under Neural Network Table.
4. From the table, I can conclude that having more layers does not always improve predictive skills. However, increading number of nodes per layer does improve preditive skills.
5. When I changed the activation function from 'relu' to 'logistics', the mean squared error decreased for almost all options of nodes, except for (1), (1,16), (1,1,1,1,1) where the mean squared error stays the same.

In [2]:
df = pd.read_csv('Waves_2023.txt', sep='\s+') 

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Thaomy\AppData\Local\Temp\ipykernel_7340\3125395425.py:1: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv('Waves_2023.txt', sep='\s+')


In [5]:
df1 = df[['WVHT', 'DPD', 'APD', 'MWD', 'WTMP']]
df1 = df1[(df1.WVHT < 99) | (df1.DPD < 99) | (df1.APD < 99) | (df1.MWD < 990)]

Xvar = df1[['DPD','APD','MWD','WTMP']].to_numpy()
yvar = np.array(df1['WVHT'])

print(Xvar)
print(yvar)

[[ 12.5    7.69 269.    15.3 ]
 [ 11.76   7.34 272.    15.3 ]
 [ 12.5    7.08 268.    15.3 ]
 ...
 [ 13.33  10.5  268.    16.8 ]
 [ 14.29  10.32 271.    16.8 ]
 [ 12.5   10.54 271.    16.9 ]]
[1.12 1.16 1.2  ... 1.79 1.66 1.7 ]


In [6]:
# I want 75% train, 10% test, 15% validate
X, X_test, y, y_test = train_test_split(Xvar, yvar, test_size=0.10)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.167)

## Baseline: Linear Model

In [8]:
linear = LinearRegression()
linear.fit(X_train, y_train)
y_train_pred = linear.predict(X_train)
y_val_pred = linear.predict(X_val)
y_test_pred = linear.predict(X_test)

print(mean_squared_error(y_train, y_train_pred))
print(mean_squared_error(y_val, y_val_pred))
print(mean_squared_error(y_test, y_test_pred))

0.19821913148311102
0.19187794204327813
0.18237403438330343


## Neural Network

In [11]:
# 3 layers, 16 nodes per layer
mlp = MLPRegressor(hidden_layer_sizes=(16,16,16,))
mlp.fit(X_train, y_train)
y_train_pred = mlp.predict(X_train)
y_val_pred = mlp.predict(X_val)
y_test_pred = mlp.predict(X_test)

print(mean_squared_error(y_train, y_train_pred))
print(mean_squared_error(y_val, y_val_pred))
print(mean_squared_error(y_test, y_test_pred))

0.1846522568122223
0.18008568828014604
0.17407571343180503


## Neural Network Table

In [12]:
nodes = [(1,), 
        (16,), 
        (1,16,), 
        (16,16,), 
        (16,4,16,), 
        (16,16,16,), 
        (16,16,16,16,), 
        (16,32,64,128),
        (1,1,1,1,1),
        (100,100,100,100,100)]

train_error = []
valid_error = []
test_error = []

for i in nodes:
    mlp = MLPRegressor(i)
    mlp.fit(X_train, y_train)
    
    y_train_pred = mlp.predict(X_train)
    y_val_pred = mlp.predict(X_val)
    y_test_pred = mlp.predict(X_test)
    
    train_err = mean_squared_error(y_train, y_train_pred)
    valid_err = mean_squared_error(y_val, y_val_pred)
    test_err = mean_squared_error(y_test, y_test_pred)
    
    train_error.append(train_err)
    valid_error.append(valid_err)
    test_error.append(test_err)

parameter = {'nodes':nodes,
             'train error':train_error,
             'validation error':valid_error,
             'test error':test_error}

In [13]:
mlp_df = pd.DataFrame(parameter)
mlp_df

,nodes,train error,validation error,test error
0,"(1,)",0.281110,0.275831,0.251345
1,"(16,)",0.206435,0.197711,0.191833
2,"(1, 16)",0.281100,0.275786,0.251403
3,"(16, 16)",0.204728,0.196756,0.190067
4,"(16, 4, 16)",0.197361,0.190345,0.184708
5,"(16, 16, 16)",0.192288,0.184293,0.177731
6,"(16, 16, 16, 16)",0.188198,0.181628,0.174021
7,"(16, 32, 64, 128)",0.181742,0.174708,0.169230
8,"(1, 1, 1, 1, 1)",0.281101,0.275792,0.251392
9,"(100, 100, 100, 100, 100)",0.168611,0.164094,0.161024


## Neural Network with Activation: 'Logistic'

In [14]:
nodes = [(1,), 
        (16,), 
        (1,16,), 
        (16,16,), 
        (16,4,16,), 
        (16,16,16,), 
        (16,16,16,16,), 
        (16,32,64,128),
        (1,1,1,1,1),
        (100,100,100,100,100)]

train_error = []
valid_error = []
test_error = []

for i in nodes:
    mlp = MLPRegressor(i, activation='logistic')
    mlp.fit(X_train, y_train)
    
    y_train_pred = mlp.predict(X_train)
    y_val_pred = mlp.predict(X_val)
    y_test_pred = mlp.predict(X_test)
    
    train_err = mean_squared_error(y_train, y_train_pred)
    valid_err = mean_squared_error(y_val, y_val_pred)
    test_err = mean_squared_error(y_test, y_test_pred)
    
    train_error.append(train_err)
    valid_error.append(valid_err)
    test_error.append(test_err)

parameter = {'nodes':nodes,
             'train error':train_error,
             'validation error':valid_error,
             'test error':test_error}

In [15]:
mlp_log_df = pd.DataFrame(parameter)
mlp_log_df

,nodes,train error,validation error,test error
0,"(1,)",0.187810,0.181842,0.172897
1,"(16,)",0.167941,0.165984,0.157416
2,"(1, 16)",0.281136,0.275742,0.251590
3,"(16, 16)",0.174084,0.171271,0.162996
4,"(16, 4, 16)",0.166418,0.162904,0.154433
5,"(16, 16, 16)",0.172077,0.170049,0.160948
6,"(16, 16, 16, 16)",0.169891,0.166697,0.160188
7,"(16, 32, 64, 128)",0.182216,0.178091,0.170225
8,"(1, 1, 1, 1, 1)",0.280963,0.275657,0.251259
9,"(100, 100, 100, 100, 100)",0.165482,0.161869,0.154383
